In [1]:
import pandas as pd
import pytz

# Data Recruitment

## Ingesting Data

In [2]:
# Creating filepath in a variable
req_path = 'data/data_reqruitment/data_requirements.csv'

# Reading csv file
df_recruitments_orignal = pd.read_csv(req_path)

# Creating copy to not edit the original
df_recruitments = df_recruitments_orignal.copy()
df_recruitments.drop(columns='Unnamed: 0', inplace=True)
df_recruitments.head()

,company,company_rating,location,job_title,job_description,salary_estimate,company_size,company_type,company_sector,company_industry,company_founded,company_revenue,dates
0,PCS Global Tech\n4.7,4.7,"Riverside, CA",Data Engineer | PAID BOOTCAMP,Responsibilities\n· Analyze and organize raw d...,"$70,000 /yr (est.)",501 to 1000 Employees,Company - Private,Information Technology,Information Technology Support Services,NaN,Unknown / Non-Applicable,2024-06-12 00:00:00-10:00
1,Futuretech Consultants LLC,NaN,"Newton, MS",Snowflake Data Engineer,My name is Dileep and I am a recruiter at Futu...,$42.50 /hr (est.),NaN,NaN,NaN,NaN,NaN,NaN,2024-06-12 00:00:00+07:00
2,Clairvoyant\n4.4,4.4,Remote,Data Engineer (MDM),Required Skills:\nMust have 5-8+ Years of expe...,$67.50 /hr (est.),51 to 200 Employees,Company - Private,Pharmaceutical & Biotechnology,Biotech & Pharmaceuticals,NaN,Unknown / Non-Applicable,2024-06-12 00:00:00-10:00
3,Apple\n4.2,4.2,"Cupertino, CA",Data Engineer,"Summary\nPosted: Dec 22, 2021\nWeekly Hours: 4...",NaN,10000+ Employees,Company - Public,Information Technology,Computer Hardware Development,1976.0,$10+ billion (USD),2024-06-12 00:00:00-05:00
4,Skytech Consultancy Services\n5.0,5.0,"Baltimore, MD",Data Engineer,Description of Work:\nTechnical experience in ...,$65.00 /hr (est.),1 to 50 Employees,Company - Public,NaN,NaN,NaN,Unknown / Non-Applicable,2024-06-12 00:00:00-04:00


## Cleaning & Transforming Data

In [3]:
# Looking how the data shape (null count, data types, columns)
df_recruitments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1555 entries, 0 to 1554
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   company           1551 non-null   object 
 1   company_rating    1358 non-null   float64
 2   location          1554 non-null   object 
 3   job_title         1554 non-null   object 
 4   job_description   1554 non-null   object 
 5   salary_estimate   1277 non-null   object 
 6   company_size      1442 non-null   object 
 7   company_type      1442 non-null   object 
 8   company_sector    1260 non-null   object 
 9   company_industry  1260 non-null   object 
 10  company_founded   1131 non-null   float64
 11  company_revenue   1442 non-null   object 
 12  dates             1555 non-null   object 
dtypes: float64(2), object(11)
memory usage: 158.1+ KB


In [4]:
# Ada beberapa error dalam kolom 'company' yang memasukan 'company rating'

# Regex untuk mendapatkan tiap elemen selain '\n'
clenased_company = df_recruitments['company'].str.findall(r'[a-zA-z0-9 ]+')

# Mengambil nama 'company' dengan asumsi index 0
def getFirstItem(item):
    try:
        fullName = str(item[0])
        return fullName
    except:
        return None

# apply dan menggantikannya kepada kolom 'company'
df_recruitments['company'] = clenased_company.apply(getFirstItem).astype('string')

In [5]:
# creating a 'numeric_salary' column for analysis
df_recruitments['numeric_salary'] = df_recruitments['salary_estimate'].str.replace(',', '').str.findall('[0-9.]+').apply(getFirstItem).astype(float)

In [6]:
# creating a 'salary_type' column to get categorical values
slrType = df_recruitments['salary_estimate'].str.findall('/[a-zA-Z]+')

def salaryType(salType):
    salary = getFirstItem(salType)

    if salary == '/yr':
        return 'yearly'
    elif salary == '/hr':
        return 'hourly'
    elif salary == '/mo':
        return 'monthly'
    else:
        return None

df_recruitments['salary_type'] = slrType.apply(salaryType).astype('string')

In [7]:
# creating a 'monthly_salary' column for easier salary comparison in analysis

def getMonthlySalary(x):
    try:
        if x['salary_type'] == 'yearly':
            salary = x['numeric_salary']/12
            salary = round(salary, 2)
            return salary
        elif x['salary_type'] == 'hourly':
            salary = x['numeric_salary'] * 173.2
            salary = round(salary, 2)
            return salary
        elif x['salary_type'] == 'monthly':
            return x['numeric_salary']
        else:
            return None
    except:
        return None


df_recruitments['monthly_salary'] = df_recruitments.apply(getMonthlySalary, axis=1)

In [8]:
# Drop values yang tidak memiliki kolom 'company'

df_recruitments.dropna(subset='company', inplace=True)

In [9]:
# Menjadikan sebagai datetime datatype
df_recruitments['dates'] = pd.to_datetime(df_recruitments['dates'], utc=True)
# Merubahnya menjadi 'Asia/Jakarta' timezone, +07:00
df_recruitments['dates'] = df_recruitments['dates'].dt.tz_convert('Asia/Jakarta')

## Data Demography

Dari data yang kita sudah bersihkan diatas, kita menemukan beberapa kolom categorical yang kita bisa analisis dengan grouping yaitu `company_type`, `company_sector`, `company_industry`. Lalu kita juga telah membersihkan kolom `company_rating` dan membentuk `monthly_salary` untuk values yang kita bisa lihat dari analisis.

In [10]:
# Melihat bentuk data secara general

df_recruitments.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1551 entries, 0 to 1554
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype                       
---  ------            --------------  -----                       
 0   company           1551 non-null   string                      
 1   company_rating    1358 non-null   float64                     
 2   location          1551 non-null   object                      
 3   job_title         1551 non-null   object                      
 4   job_description   1551 non-null   object                      
 5   salary_estimate   1274 non-null   object                      
 6   company_size      1439 non-null   object                      
 7   company_type      1439 non-null   object                      
 8   company_sector    1257 non-null   object                      
 9   company_industry  1257 non-null   object                      
 10  company_founded   1128 non-null   float64                     
 11  company_r

In [11]:
# Menunjukan masing-masing columns
df_recruitments.columns

Index(['company', 'company_rating', 'location', 'job_title', 'job_description',
       'salary_estimate', 'company_size', 'company_type', 'company_sector',
       'company_industry', 'company_founded', 'company_revenue', 'dates',
       'numeric_salary', 'salary_type', 'monthly_salary'],
      dtype='object')

Mencari unique values untuk masing-masing categorical variables

In [12]:
df_recruitments['company_type'].unique()

array(['Company - Private', nan, 'Company - Public', 'Hospital',
       'Contract', 'Subsidiary or Business Segment',
       'Nonprofit Organization', 'Private Practice / Firm', 'Government',
       'Self-employed', 'College / University', 'Unknown',
       'School / School District'], dtype=object)

In [13]:
df_recruitments['company_sector'].unique()

array(['Information Technology', nan, 'Pharmaceutical & Biotechnology',
       'Management & Consulting', 'Healthcare',
       'Human Resources & Staffing', 'Manufacturing',
       'Financial Services', 'Hotels & Travel Accommodation',
       'Transportation & Logistics', 'Retail & Wholesale',
       'Arts, Entertainment & Recreation',
       'Government & Public Administration', 'Aerospace & Defense',
       'Personal Consumer Services', 'Media & Communication',
       'Energy, Mining & Utilities', 'Insurance',
       'Construction, Repair & Maintenance Services', 'Agriculture',
       'Education', 'Real Estate', 'Legal', 'Restaurants & Food Service',
       'Nonprofit & NGO', 'Telecommunications'], dtype=object)

In [14]:
print(df_recruitments['company_industry'].nunique())
# memiliki 83 unique values

df_recruitments['company_industry'].unique()

83


array(['Information Technology Support Services', nan,
       'Biotech & Pharmaceuticals', 'Computer Hardware Development',
       'Research & Development', 'Health Care Services & Hospitals',
       'Internet & Web Services', 'HR Consulting',
       'Consumer Product Manufacturing',
       'Enterprise Software & Network Solutions',
       'Investment & Asset Management', 'Accounting & Tax',
       'Business Consulting', 'Financial Transaction Processing',
       'Staffing & Subcontracting', 'Banking & Lending',
       'Travel Agencies', 'Airlines, Airports & Air Transportation',
       'Automotive Parts & Accessories Stores', 'Sports & Recreation',
       'National Agencies', 'Aerospace & Defense', 'Software Development',
       'Beauty & Wellness', 'Publishing', 'Food & Beverage Manufacturing',
       'Energy & Utilities', 'Advertising & Public Relations',
       'Insurance Carriers', 'Wholesale',
       'Department, Clothing & Shoe Stores', 'Machinery Manufacturing',
       'Archite

Mencari rata-rata untuk Number variables. (int & float)

In [15]:
# Untuk company rating
print(df_recruitments['company_rating'].mean())

3.925625920471281


In [16]:
# Max dan Mean untuk salary
print(df_recruitments['monthly_salary'].max())
print(df_recruitments['monthly_salary'].mean())

36000.0
9821.691106750392


menggunaka describe untuk analisis kolom secara general.

In [17]:
# Untuk analisis kolom numerical
df_recruitments.describe()

,company_rating,company_founded,numeric_salary,monthly_salary
count,1358.000000,1128.000000,1274.000000,1274.000000
mean,3.925626,1975.929078,98180.690424,9821.691107
std,0.535826,51.530079,50629.066214,2893.404988
min,1.000000,1636.000000,22.500000,3897.000000
25%,3.600000,1964.750000,84366.000000,7879.332500
50%,3.900000,1997.000000,102802.000000,9347.915000
75%,4.200000,2009.000000,125000.000000,11243.605000
max,5.000000,2022.000000,341000.000000,36000.000000


In [18]:
# Untuk analisis kolom dengan datatype object
df_recruitments.describe(include=object)

,location,job_title,job_description,salary_estimate,company_size,company_type,company_sector,company_industry,company_revenue
count,1551,1551,1551,1274,1439,1439,1257,1257,1439
unique,418,780,1549,953,8,12,25,83,10
top,Remote,Data Engineer,Looking for a Data Logging Engineer\nWhat you ...,"$120,000 /yr (est.)",10000+ Employees,Company - Private,Information Technology,Information Technology Support Services,Unknown / Non-Applicable
freq,234,356,2,17,433,726,507,215,551


### Company Rating 
Melakukan agregasi nilai rating, dengan mengroup kolom kategorical kita 'company_type', 'company_industry', dan 'company_sector'.

In [19]:
df_recruitments.groupby('company_type')['company_rating'].agg(['mean', 'min', 'max', 'count']).sort_values(by='mean', ascending=False)

,mean,min,max,count
company_type,,,,
College / University,4.193333,3.9,4.5,15
Private Practice / Firm,4.030000,3.4,4.5,10
Unknown,4.000000,3.1,4.6,8
Company - Private,3.988143,2.0,5.0,700
Company - Public,3.891813,1.3,5.0,513
Nonprofit Organization,3.729545,2.2,4.6,44
Subsidiary or Business Segment,3.721875,2.9,4.7,32
School / School District,3.700000,3.7,3.7,1
Contract,3.666667,1.0,5.0,12


In [20]:
df_recruitments.groupby('company_sector')['company_rating'].agg(['mean', 'min', 'max', 'count']).sort_values(by='mean', ascending=False)

,mean,min,max,count
company_sector,,,,
Human Resources & Staffing,4.192857,3.4,5.0,28
Hotels & Travel Accommodation,4.075000,3.3,4.5,4
Information Technology,4.050000,1.3,5.0,500
"Arts, Entertainment & Recreation",4.046154,3.1,5.0,13
Aerospace & Defense,3.997674,3.5,4.8,43
Education,3.921429,3.0,4.5,28
Real Estate,3.910000,3.5,4.7,10
Pharmaceutical & Biotechnology,3.850000,2.8,4.6,28
Management & Consulting,3.850000,2.4,4.8,72


In [21]:
df_recruitments.groupby(['company_sector','company_industry'])['company_rating'].agg(['mean', 'min', 'max', 'count']).sort_values(by='mean', ascending=False)

mean  \
company_sector                              company_industry                                     
Retail & Wholesale                          Consumer Electronics & Appliances Stores  4.500000   
                                            Automotive Parts & Accessories Stores     4.333333   
Arts, Entertainment & Recreation            Gambling                                  4.300000   
Human Resources & Staffing                  Staffing & Subcontracting                 4.228571   
Media & Communication                       Publishing                                4.200000   
...                                                                                        ...   
Healthcare                                  Medical Testing & Clinical Laboratories   3.100000   
Nonprofit & NGO                             Religious Institutions                    3.100000   
Management & Consulting                     Building & Personnel Services             3.100000   
Retail & Wholesale                          Other Retail Stores                       2.900000   
Construction, Repair & Maintenance Services Vehicle Repair & Maintenance              2.400000   

                                                                                      min  \
company_sector                              company_industry                                
Retail & Wholesale                          Consumer Electronics & Appliances Stores  4.5   
                                            Automotive Parts & Accessories Stores     3.9   
Arts, Entertainment & Recreation            Gambling                                  3.9   
Human Resources & Staffing                  Staffing & Subcontracting                 3.6   
Media & Communication                       Publishing                                4.1   
...                                                                                   ...   
Healthcare                                  Medical Testing & Clinical Laboratories   3.1   
Nonprofit & NGO                             Religious Institutions                    3.1   
Management & Consulting                     Building & Personnel Services             2.4   
Retail & Wholesale                          Other Retail Stores                       2.7   
Construction, Repair & Maintenance Services Vehicle Repair & Maintenance              2.4   

                                                                                      max  \
company_sector                              company_industry                                
Retail & Wholesale                          Consumer Electronics & Appliances Stores  4.5   
                                            Automotive Parts & Accessories Stores     4.8   
Arts, Entertainment & Recreation            Gambling                                  5.0   
Human Resources & Staffing                  Staffing & Subcontracting                 4.5   
Media & Communication                       Publishing                                4.3   
...                                                                                   ...   
Healthcare                                  Medical Testing & Clinical Laboratories   3.1   
Nonprofit & NGO                             Religious Institutions                    3.1   
Management & Consulting                     Building & Personnel Services             3.8   
Retail & Wholesale                          Other Retail Stores                       3.1   
Construction, Repair & Maintenance Services Vehicle Repair & Maintenance              2.4   

                                                                                      count  
company_sector                              company_industry                                 
Retail & Wholesale                          Consumer Electronics & Appliances Stores      1  
                                            Automotive Parts & Accessories Stores         3  
Arts, Entertainment

In [22]:
# Memastikan bahwa count dari 'company_industry' memiliki minimum count > 5
rec_industry = df_recruitments.groupby(['company_sector','company_industry'])['company_rating'].agg(['mean', 'min', 'max', 'count']).sort_values(by='mean', ascending=False)
rec_industry[rec_industry['count'] > 5]

mean  \
company_sector                              company_industry                                    
Human Resources & Staffing                  Staffing & Subcontracting                4.228571   
                                            HR Consulting                            4.180952   
Arts, Entertainment & Recreation            Sports & Recreation                      4.157143   
Information Technology                      Enterprise Software & Network Solutions  4.150962   
Education                                   Colleges & Universities                  4.135294   
Information Technology                      Computer Hardware Development            4.098765   
                                            Software Development                     4.080952   
Government & Public Administration          National Agencies                        4.069231   
Information Technology                      Information Technology Support Services  4.034597   
Aerospace & Defense                         Aerospace & Defense                      3.997674   
Management & Consulting                     Business Consulting                      3.950000   
Financial Services                          Financial Transaction Processing         3.940000   
Construction, Repair & Maintenance Services Architectural & Engineering Services     3.930000   
Manufacturing                               Consumer Product Manufacturing           3.923077   
Financial Services                          Investment & Asset Management            3.918519   
Real Estate                                 Real Estate                              3.910000   
Financial Services                          Accounting & Tax                         3.886667   
Manufacturing                               Food & Beverage Manufacturing            3.877778   
Pharmaceutical & Biotechnology              Biotech & Pharmaceuticals                3.861538   
Management & Consulting                     Research & Development                   3.850000   
Information Technology                      Internet & Web Services                  3.848387   
Media & Communication                       Advertising & Public Relations           3.827273   
Energy, Mining & Utilities                  Energy & Utilities                       3.806667   
Financial Services                          Banking & Lending                        3.783607   
Manufacturing                               Machinery Manufacturing                  3.750000   
Insurance                                   Insurance Carriers                       3.740476   
Manufacturing                               Transportation Equipment Manufacturing   3.672727   
Retail & Wholesale                          Wholesale                                3.663636   
Restaurants & Food Service                  Restaurants & Cafes                      3.637500   
Government & Public Administration          State & Regional Agencies                3.587500   
Healthcare                                  Health Care Services & Hospitals         3.509211   
Education                                   Education & Training Services            3.500000   
Transportation & Logistics                  Shipping & Trucking                      3.342857   

                                                                                     min  \
company_sector                              company_industry                               
Human Resources & Staffing                  Staffing & Subcontracting                3.6   
                                            HR Consulting                            3.4   
Arts, Entertainment & Recreation            Sports & Recreation                      3.4   
Information Technology                      Enterprise Software & Network Solutions  2.9   
Education                                   Colleges & Universities                  3.6   
Information Technology                      Compu

Dari hasil fungsi agregasi diatas, kita bisa melihat bahwa melalui kategori `company_type` yang memiliki nilai rata-rata rating yang paling tinggi merupakan kelompok 'College/Unviersity'. Namun perlu dilihat bahwa count dari kategori tersebut bisa dibilang sedikit. Perlu dilihat 'Company - Private' dan 'Company - Public' memiliki rating yang yang relatif tinggi 3.9 dan 3.8 dengan jumlah count yang banyak (700 & 500).

Bila kita mengagregasi dengan kolom kategori `company_sector`, kita bisa melihat bahwa kategori 'Human resources & Staffing' merupakan kategori dengan nilai rating tertinggi. Namun, sekali lagi terdapat kategori yang memiliki jumlah yang banyak (500) dan memiliki angka signifikan (4.0) yaitu 'Information Technology'.

Hal ini konsisten dengan `company_industry` sebagai sub-kategori dari `company_sector`. Walaupun bila di agregat secara langsung kita menumukan rata-rata tertinggi berada di kategori 'consumer electronics' & 'automotive store', mereka memiliki jumlah yang sedikit (kurang dari lima). Ketika kita melakukan filtering, baru hasil konsisten dengan dipuncaki `company_industry` yang memiliki kategori `company_sector` 'Human Resources & Staffing` dan `Information Technology`.

### Monthy Salary

Melakukan agregasi nilai 'monthly_salary', dengan mengroup dalam kolom kategorikal kita 'company_type', 'company_industry', dan 'company_sector'.

In [23]:
df_recruitments.groupby('company_type')['monthly_salary'].agg(['mean', 'min', 'max', 'count']).sort_values(by='mean', ascending=False)

,mean,min,max,count
company_type,,,,
Self-employed,11875.000000,10625.00,12500.00,3
Company - Public,10245.430130,4000.00,28416.67,463
Company - Private,9667.472926,3897.00,27712.00,598
Unknown,9348.728333,5303.92,13261.17,12
Private Practice / Firm,9023.546667,5657.67,16666.67,9
Government,8902.362143,6422.58,12142.00,14
Subsidiary or Business Segment,8881.402069,5238.42,14202.40,29
College / University,8607.819091,6144.17,11620.75,11
Nonprofit Organization,8315.824324,5385.33,12050.00,37


In [24]:
df_recruitments.groupby('company_sector')['monthly_salary'].agg(['mean', 'min', 'max', 'count']).sort_values(by='mean', ascending=False)

,mean,min,max,count
company_sector,,,,
Telecommunications,11864.648889,8071.33,16550.00,9
Hotels & Travel Accommodation,11239.585000,8500.00,12291.67,4
Information Technology,10482.329591,3897.00,28416.67,391
Human Resources & Staffing,10176.908276,4760.42,16666.67,29
Management & Consulting,10114.562985,4330.00,20208.33,67
Pharmaceutical & Biotechnology,9913.469167,5306.67,17500.00,24
Media & Communication,9879.635172,6188.17,14787.50,29
Aerospace & Defense,9428.791875,6183.50,15833.33,32
Restaurants & Food Service,9425.492727,4854.42,12788.17,11


In [25]:
df_recruitments.groupby(['company_sector','company_industry'])['monthly_salary'].agg(['mean', 'min', 'max', 'count']).sort_values(by='max', ascending=False)

mean  \
company_sector          company_industry                                         
Information Technology  Computer Hardware Development             11232.154828   
                        Enterprise Software & Network Solutions   11605.577733   
Management & Consulting Security & Protective                     14784.982000   
                        Research & Development                     9763.102222   
Information Technology  Internet & Web Services                   11709.809000   
...                                                                        ...   
Education               Primary & Secondary Schools                6379.750000   
Healthcare              Medical Testing & Clinical Laboratories    5303.920000   
Management & Consulting Membership Organizations                           NaN   
Retail & Wholesale      Consumer Electronics & Appliances Stores           NaN   
                        Gift, Novelty & Souvenir Stores                    NaN   

                                                                      min  \
company_sector          company_industry                                    
Information Technology  Computer Hardware Development             5113.83   
                        Enterprise Software & Network Solutions   6622.50   
Management & Consulting Security & Protective                     9133.25   
                        Research & Development                    4330.00   
Information Technology  Internet & Web Services                   6441.67   
...                                                                   ...   
Education               Primary & Secondary Schools               6379.75   
Healthcare              Medical Testing & Clinical Laboratories   5303.92   
Management & Consulting Membership Organizations                      NaN   
Retail & Wholesale      Consumer Electronics & Appliances Stores      NaN   
                        Gift, Novelty & Souvenir Stores               NaN   

                                                                       max  \
company_sector          company_industry                                     
Information Technology  Computer Hardware Development             28416.67   
                        Enterprise Software & Network Solutions   21166.67   
Management & Consulting Security & Protective                     20208.33   
                        Research & Development                    19791.67   
Information Technology  Internet & Web Services                   19460.00   
...                                                                    ...   
Education               Primary & Secondary Schools                6379.75   
Healthcare              Medical Testing & Clinical Laboratories    5303.92   
Management & Consulting Membership Organizations                       NaN   
Retail & Wholesale      Consumer Electronics & Appliances Stores       NaN   
                        Gift, Novelty & Souvenir Stores                NaN   

                                                                  count  
company_sector          company_industry                                 
Information Technology  Computer Hardware Development                58  
                        Enterprise Software & Network Solutions      75  
Management & Consulting Security & Protective                         5  
                        Research & Development                        9  
Information Technology  Internet & Web Services                      50  
...                                                                 ...  
Education               Primary & Secondary Schools                   1  
Healthcare              Medical Testing & Clinical Laboratories       1  
Management & Consulting Membership Organizations                      0  
Retail & Wholesale      Consumer Electronics & Appliances Stores      0  
                        Gift, Novelty & Souvenir Stores               0  

[83 rows x 4 columns]

Dari analisis diatas kita menemukan bahwa walaupun dalam `company_type` terdapat 'Self-employeed' merupakan tertinggi, namun perlu dilihat karena jumlah countnya hanyalah berjumlah 3. Lebih akurat jika kita lihat bahwa 'Company Public' dan 'Company Private' merupakan rata-rata tertinggi.

Fenomena ini kita lihat lagi dalam `company_sector` dan `company_industry`. Dimana dipuncaki oleh 'telecomunications' dan 'hotel & travel acommodation', tetapi berjumlah sedikit. Kita bisa melihat 'information technology' sebagai kategori yang memiliki rata-rata `monthly_salary` tertinggi. Hal ini didukung dengan melihat `company_industy` yang memiliki `company_sector` banyak dalam 'information technology'.